In [1]:
import import_ipynb
from CustomDataset import ControlsDataset

import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Ignore warnings
import warnings
import time
warnings.filterwarnings("ignore")


from tqdm.notebook import tqdm_notebook

importing Jupyter notebook from CustomDataset.ipynb


In [2]:
class Trainer():
    def __init__(self, device, model, dataset, optimizer, criterion):
        self.device = device
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.dataset = dataset
        
    def initializeEpoch(self):
        self.summation = 0
        self.val_summation = 0
        self.validation_training = enumerate(self.dataset.validloader)
    
    def fit(self, epochs, report_period):
        save_path = "snapshots/{}_{:.5f}_weights.pt"
        iters_trained = []
        training_losses = []
        validation_losses = []
        count = 0
        
        counts,values = self.label_distribution()
        
        self.inverse = max(counts)/counts
        self.values = values
        
        #self.inverse = [10,10,10,10,10,1,10,10,10,10]
        #self.inverse = [1,1,1,1,1,1,1,1,1,1]
        
        print(self.inverse)
        print(self.values)
        
        plt.show()
        
        for epoch in range(epochs):
            self.initializeEpoch()
            for i_batch, sampled_batch in tqdm_notebook(enumerate(self.dataset.dataloader),
                                                       total=len(self.dataset.dataloader)):
                #print(torch.cuda.max_memory_cached(self.device))
                torch.cuda.empty_cache()
                #print(torch.cuda.max_memory_cached(self.device))
                self.model.train()
                #inputs and forward pass
                images = sampled_batch['image'].to(self.device).float()
                #myimage = images[0].cpu().numpy().transpose((1,2,0))
                #plt.imshow(myimage)
                controls = sampled_batch['control'].to(self.device).float()
                controls = torch.flatten(controls)
                print(controls)
                
                #backwards pass
                self.optimizer.zero_grad()
                
                #print(torch.cuda.max_memory_cached(self.device))
                prediction = self.model(images)
                #print(torch.cuda.max_memory_cached(self.device))
                
                prediction = torch.flatten(prediction)
                
                #calculate loss
                loss = self.criterion(prediction, controls)
                weights = [self.calculate_weight(control) for control in controls]
                weights = torch.tensor(weights).to(self.device).float()
                #loss = torch.mul(loss,weights)
                #loss = torch.mean(loss)
                
                loss.backward()
                self.optimizer.step()
                
                #get batch losses
                val_i,batch = self.validationBatch()
                val_loss = self.score(batch)
                #val_loss = val_loss.data
                self.summation += float(loss)
                self.val_summation += float(val_loss)
                
                if i_batch % report_period == 0:
                    iters_trained.append(count)
                    average_loss = round(self.summation/float(i_batch+1),5)
                    average_val_loss = round(self.val_summation/float(i_batch+1),5)
                    training_losses.append(average_loss)
                    validation_losses.append(average_val_loss)
                count += 1
                #print(torch.cuda.max_memory_cached(self.device))
                    
            print("Epoch: "+str(epoch))
            print("Training Loss: "+str(average_loss))
            print("Validation Loss: "+str(average_val_loss))
            #self.model.save_weights(self.optimizer, epoch, save_path.format(count,average_loss))
            #torch.save(self.model, save_path.format(count,average_loss))  
            plt.plot(iters_trained,training_losses, label="training")
            plt.plot(iters_trained,validation_losses, label="validation")
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
            plt.show() 
            
        
        
    def validationBatch(self):
        try:
            val_i,batch = next(self.validation_training)
        except StopIteration:
            self.validation_training = enumerate(self.dataset.validloader)
            val_i,batch = next(self.validation_training)
        return val_i,batch
        
                
    def score(self, sampled_batch):
        self.model.eval()
        #inputs and forward pass
        images = sampled_batch['image'].to(self.device).float()
        controls = sampled_batch['control'].to(self.device).float()
        controls = torch.flatten(controls)
        print(controls)

        #backwards pass
        self.optimizer.zero_grad()
        #print(torch.cuda.max_memory_cached(self.device))
        prediction = self.model(images)
        #print(torch.cuda.max_memory_cached(self.device))
        prediction = torch.flatten(prediction)

        #calculate loss
        loss = self.criterion(prediction, controls)
        weights = [self.calculate_weight(control) for control in controls]
        weights = torch.tensor(weights).to(self.device).float()
        #loss = torch.mul(loss,weights)
        #loss = torch.mean(loss)
        torch.cuda.empty_cache()
        #print(torch.cuda.max_memory_cached(self.device))
        return loss
    
    def label_distribution(self):
        histogram = plt.hist(self.dataset.labels.dataframe["Angle"])
        count = histogram[0]
        values = histogram[1]
        return count,values
    
    def calculate_weight(self,control):
        for i in range(len(self.inverse)):
            if control >= self.values[i] and control < self.values[i+1]:
                return self.inverse[i]
        return self.inverse[-1]